In [1]:
import numpy as np
import pickle
from tqdm.notebook import tqdm as tqdm_notebook
import re
import collections
import glob
from transformers import DistilBertTokenizer, DistilBertModel
import sys
import time
import requests
from IPython.display import display, HTML
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

sys.path.insert(0, '../src/models/')
sys.path.insert(0, '../src/features/')

from predict_model import loadBERT
from predict_model import SpanPredictor as classify
from build_features import text_cleaner, DuckDuckGo_Java, Bing_HTML, colorize_prediction

%matplotlib inline

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- T

In [2]:
model = loadBERT("../models/", 'saved_weights_inf_FIXED_boot_beta80.pt')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

CPU Success


In [3]:
plants_dict = collections.defaultdict(list)
root = '../data/description/'
# Load the pickle list
data_files = glob.glob(root+ 'description*PLANTS.pkl')
for data_file in data_files:
    dict_ = pickle.load(open(data_file, 'rb'))
    for key, value in dict_.items():
        plants_dict[key] += value
    

# Order the dictionary based on the list length
plants_dict = collections.OrderedDict(sorted(plants_dict.items(), key= lambda x: len(x[1]), reverse=True))
# Correct first key
plants_dict['Poa'] = plants_dict.pop('oa')
# Get keys
plants = [key for key in plants_dict.keys()]

# FOR THE BIRD TESTING
#bird_descriptions_dict = pickle.load(open('../data/description/04_TRAIN_0-1881_BIRDS.pkl', 'rb'))
#birds = [key for key in bird_descriptions_dict.keys()]

In [7]:
spps = ['Wodyetia bifurcata']

In [4]:
driver = webdriver.Safari()

In [ ]:
#DuckDuckGo_Java('Robert van de VLasakker', driver=driver)

### URLS

In [8]:
# DEBUGGING
data_links = collections.defaultdict(list)
# Init driver
#driver = webdriver.Safari()

queries = ['description', 'diagnosis', '', 'attributes', 'captions']

for species in tqdm_notebook(spps):
    # Empty list
    search_links = []
    
    for query in queries:
        # create query
        species_q = species.replace(' ', '+')
        species_q = f'"{species_q}"+{query}'
        try:
            search_links += DuckDuckGo_Java(species_q, 
                                            driver=driver)
            search_links += Bing_HTML(species_q)
         # Skip connection timeout
        except:
            continue
    # Drop duplicates
    search_links = list(set(search_links))
    if not search_links:
        print('empty')
    # DEBUGGING
    data_links[species] += search_links

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#with open('../data/description/01_URLS_0020000-0025000_PLANTS.pkl', 'wb') as f:
#    pickle.dump(data_links, f)

In [11]:
#data_links

### TEXT

In [12]:
#URLS = pickle.load(open('../data/description/01_URLS_0015000-0020000_PLANTS.pkl', 'rb'))
URLS = data_links

In [13]:
sentence_list = collections.defaultdict(list)

species = list(URLS.keys())

for species in tqdm_notebook(species, desc='Species'):
    for URL in URLS[species]:
        # Skip google archives
        if 'google' in URL:
            continue
        # PDF and TXT
        if URL.endswith('txt') or URL.endswith('pdf'):
            continue
        # Skip Plants of the world (already done)
        if 'powo' in URL:
            continue
        try:
            page = requests.get(URL, timeout=5)
            # Skip PDF files for now
            if page.headers['Content-Type'].startswith('application/pdf'):
                continue
            # Soup the result
            soup = BeautifulSoup(page.content, "lxml", from_encoding="iso-8859-1")    
            # Skip Embedded PDF's
            if 'pdf' in soup.title.text.lower():
                continue
            # Check if species exists somewhere within title
            if bool(set(species.split()).intersection(soup.title.text.split())):
                # Get text
                #dirty_text = soup.get_text(". ", strip=True)
                dirty_text = soup.get_text(" ", strip=False).replace('\n', '.')
                # Clean and break into sents
                sentences = text_cleaner(dirty_text)
                # Append
                sentence_list[species].append(sentences)
        except:
            continue

Species:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
x = []
for y in sentence_list[species]:
    x.extend(y)
#sentence_list

In [21]:
len(x)

2097

In [ ]:
#with open('../data/description/02_SENTS_0015000-0020000_PLANTS.pkl', 'wb') as f:
#    pickle.dump(sentence_list, f)

# Classify

In [22]:
'''
# Load the pickle list
datalist = glob.glob('../data/description/02_SENTS_0015_PLANTS.pkl')
# Init list
sentence_list = {}
# Loop over the pickles
for data in datalist:
    # Open the pickles
    datadict = pickle.load(open(data, 'rb'))
    # Update
    sentence_list.update(datadict)
'''

sentence_list = pickle.load(open('../data/description/02_SENTS_0015000-0020000_PLANTS.pkl', 'rb'))

In [24]:
descriptions = collections.defaultdict(list)
species_list = list(sentence_list.keys())

for species in tqdm_notebook(species_list, desc='Species'):
    for text in sentence_list[species]:
        for sentence in text:
            if classify(sentence, model=model):
                descriptions[species].append(sentence)

        

Species:   0%|          | 0/4248 [00:00<?, ?it/s]

In [25]:
with open('../data/description/03_DESC_0015000-0020000_PLANTS.pkl.pkl', 'wb') as f:
    pickle.dump(descriptions, f)

# Add Know Data

In [8]:
# Load the pickle list
datalist = glob.glob('../data/description/03_DESC*PLANTS.pkl')
# Init list
description_list = {}
# Loop over the pickles
for data in datalist:
    # Open the pickles
    datadict = pickle.load(open(data, 'rb'))
    # Update
    description_list.update(datadict)

In [9]:
len(description_list.keys())

14557

In [10]:
# Load the pickle list
datalist = glob.glob('../data/description/description*.pkl')
# Init list
known_list = {}
# Loop over the pickles
for data in datalist:
    # Open the pickles
    datadict = pickle.load(open(data, 'rb'))
    # Update
    known_list.update(datadict)

In [14]:
traindata = collections.defaultdict(list)

species = description_list.keys()

for species in description_list.keys():
    for sentence in description_list[species]:
        if len(sentence.split()) < 3:
            continue
        traindata[species].append(sentence)
    for (sentence, URL) in known_list[species]:
        if len(sentence.split()) < 3:
            continue
        traindata[species].append(sentence)

In [15]:
len(traindata.keys())

14557

In [16]:
with open('../data/description/04_DESC_0000000-0014557_PLANTS.pkl', 'wb') as f:
    pickle.dump(traindata, f)